In [ ]:
!pip install pandas

In [1]:
import re as re
import pandas as pd

In [2]:
fb = pd.read_csv("fb-israeli-pages-with-text.csv.gz") #3,931,344 מס שורות

/tmp/ipykernel_665/1895396563.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  fb = pd.read_csv("fb-israeli-pages-with-text.csv.gz") #3,931,344 מס שורות


In [3]:
# ניקוי דאטה
fb = fb.drop(columns=['link', 'caption', 'account.handle','comments', 'likes','id','description','title'])
fb = fb.dropna()
fb = fb.drop_duplicates()#3,926,60

In [4]:
tweets2018 = pd.read_csv("2018tweets.csv")

In [5]:
tweets2018 = tweets2018.drop(columns=['author_id', 'id'])

In [6]:
# 2. פונקציה לבדיקת מילים באנגלית
def contains_english(text):
    if isinstance(text, str):  # בדיקה אם הערך הוא מחרוזת
        return bool(re.search(r"[a-zA-Z]", text))
    return False
#מכיל שורות 1,576,908

# 3. פונקציה לבדיקה אם הטקסט מכיל ערבית
def contains_arabic(text):
    if isinstance(text, str):  # בדיקה אם הערך הוא מחרוזת
        return bool(re.search(r"[\u0600-\u06FF]", text))  # טווח Unicode לערבית
    return False

def clean_text(text):
    import re
    text = re.sub(r"http\S+", "", text)  # הסרת קישורים
    text = re.sub(r"[^\w\s]", "", text)  # הסרת סימני פיסוק
    return text


In [5]:
fb = fb[~fb['message'].apply(contains_arabic)]
fb_hebrew =  fb[~fb['message'].apply(contains_english)]
fb_hebrew['message'] = fb_hebrew['message'].apply(clean_text)
fb.shape[0]
fb_hebrew.shape[0] #1,454,719

/tmp/ipykernel_665/184419762.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_hebrew['message'] = fb_hebrew['message'].apply(clean_text)


1454719

In [7]:
tweets2018 = tweets2018[~tweets2018['text'].apply(contains_arabic)]
tweets2018 =  tweets2018[~tweets2018['text'].apply(contains_english)]
tweets2018['text'] = tweets2018['text'].apply(clean_text)
tweets2018 = tweets2018[tweets2018['text'].str.len() > 2] 
tweets2018 = tweets2018[tweets2018['text'].str.len() < 750]
tweets2018 = tweets2018.drop_duplicates(subset=['text'])
tweets2018.shape #201,996

(201996, 3)

In [6]:
fb_hebrew = fb_hebrew[fb_hebrew['message'].str.len() > 2] #1,384,575
fb_hebrew = fb_hebrew[fb_hebrew['message'].str.len() < 750] #1,384,575
fb_hebrew.shape[0]#1,383,050

1383050

In [7]:
fb_hebrew.to_csv('fb_hebrew', index=False, encoding='utf-8-sig')
print("הנתונים הנקיים נשמרו בקובץ fb_hebrew.csv")

הנתונים הנקיים נשמרו בקובץ fb_hebrew.csv


In [8]:
tweets2018.to_csv('tweets2018_clean', index=False, encoding='utf-8-sig')

In [4]:
fb_1 = pd.read_csv("fb_hebrew") #3,931,344 מס שורות

In [ ]:
fb = pd.read_csv("fb-israeli-pages-with-text.csv.gz") #3,931,344 מס שורות

In [36]:
def contains_only_english(text):
    if isinstance(text, str):  # בדיקה אם הערך הוא מחרוזת
        # אם לא נמצאו אותיות בעברית, נשמור את השורה
        return not bool(re.search(r"[\u0590-\u05FF]", text))
    return False  # עבור ערכים שאינם מחרוזת
fb_english = fb[fb["message"].apply(contains_only_english)]
fb_english['message'] = fb_english['message'].apply(clean_text)
fb_english = fb_english[fb_english['message'].str.len() < 750]
fb_english = fb_english[fb_english['message'].str.len() > 1 ]

fb_english_100 = fb_english[:100]

from transformers import pipeline
# טעינת המודל לניתוח סנטימנט
sentiment_analyzer = pipeline("sentiment-analysis")
# פונקציה לניתוח סנטימנט
def analyze_sentiment(text):
    try:
        result = sentiment_analyzer(text[:512])  # מגבלת אורך של 512 תווים
        return result[0]['label'], result[0]['score']
    except Exception as e:
        return "ERROR", 0.0
# הוספת עמודות סנטימנט לדאטה
fb_english_100['sentiment'], fb_english_100['score'] = zip(*fb_english_100['message'].apply(analyze_sentiment))
fb_english_100.head()
fb_english_100.to_csv("english_only_with_sentiment.csv", index=False)

/tmp/ipykernel_98/2529199437.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  fb = pd.read_csv("fb-israeli-pages-with-text.csv.gz") #3,931,344 מס שורות
/tmp/ipykernel_98/2529199437.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fb_english['message'] = fb_english['message'].apply(clean_text)
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
/tmp/ipykernel_98/2529199437.py:25: SettingWithCopyWarning: 
A value is trying to

In [ ]:
classifier = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")
# פונקציה לסיווג אם הפוסט פיננסי
def is_financial_classification(text)
    try:
        result = classifier(text[:512])
        return "finance" in result[0]['label'].lower()  # בדיקה אם המודל מזהה את זה כפיננסי
    except:
        return False
# סינון פוסטים פיננסיים
financial_posts = fb_english[fb_english['message'].apply(is_financial_classification)]
print(financial_posts.head())